In [ ]:
import numpy as np
from enum import Enum
import numpy as np
import pandas as pd
import talib

In [ ]:

class ComparisonType(Enum):
    ABOVE = "above"
    BELOW = "below"
    CROSSED_UP = "crossed_up"
    CROSSED_DOWN = "crossed_dn"
    EQUALS = "equals"
    GREATER_EQUAL = "greater_equal"
    LESS_EQUAL = "less_equal"

query = """
Close above EMA_21
RSI_14 below 30
Volume above 1500
"""

# Split into words
words = query.split()

# Remove words that are in ComparisonType
filtered_words = [word for word in words if word.lower() not in {c.value for c in ComparisonType}]

# Prepare indiSet
indiSet = []

for word in filtered_words:
    if "_" in word:  # e.g., EMA_21
        parts = word.split("_")
        indicator = parts[0].upper()
        period = int(parts[1]) if parts[1].isdigit() else None
        indiSet.append([indicator, period, None, None])
    elif word.isdigit():  # just a number
        continue  # skip standalone numbers
    else:  # single word indicator like 'Close', 'Volume'
        indiSet.append([word.upper(), None, None, None])

indiSet = np.array(indiSet, dtype=object)

print(indiSet)

In [ ]:


# Example DataFrame
ohlc_ta = pd.read_csv( "xauusdm15.filtered.csv" )


# Indicator set as 2D NumPy array
indiSet = np.array([
    ["EMA", 21, None, None],
    ["RSI", 14, None, None],
    ["SMA", 10, None, None],
], dtype=object)

# Loop over indicators
for row in indiSet:
    name, p1, p2, p3 = row   # unpack values
    func = getattr(talib, name)   # get the TA-Lib function by name

    # Build kwargs depending on which params are present
    kwargs = {}
    if p1 is not None: kwargs["timeperiod"] = int(p1)
    if p2 is not None: kwargs["fastperiod"] = int(p2)
    if p3 is not None: kwargs["signalperiod"] = int(p3)

    # Apply indicator
    ohlc_ta[f"{name}_{p1}"] = func(ohlc_ta["Close"], **kwargs)

ohlc_ta.T


In [ ]:
import numpy as np
import pandas as pd
import talib
from enum import Enum

# --- Step 0: ComparisonType ---
class ComparisonType(Enum):
    ABOVE = "above"
    BELOW = "below"
    CROSSED_UP = "crossed_up"
    CROSSED_DOWN = "crossed_dn"
    EQUALS = "equals"
    GREATER_EQUAL = "greater_equal"
    LESS_EQUAL = "less_equal"

# --- Step 1: Load Data ---
ohlc_ta = pd.read_csv("xauusdm15.filtered.csv")

# --- Step 2: Parse Query into indiSet ---
query = """
Close above EMA_21
RSI_14 below 30
Volume above 1500
"""

words = query.split()
indiSet = []

for word in words:
    # skip comparison words
    if word.lower() in {c.value for c in ComparisonType}:
        continue

    # handle indicator with period like EMA_21
    if "_" in word:
        parts = word.split("_")
        indicator = parts[0].upper()
        period = int(parts[1]) if parts[1].isdigit() else None
        indiSet.append([indicator, period, None, None])
    # skip standalone numbers
    elif word.isdigit():
        continue
    # single-word indicator
    else:
        indiSet.append([word.upper(), None, None, None])

indiSet = np.array(indiSet, dtype=object)

# --- Step 3: Apply Indicators Dynamically ---
for row in indiSet:
    name, p1, p2, p3 = row

    # Skip if function does not exist in TA-Lib
    if not hasattr(talib, name):
        print(f"Skipping unknown indicator: {name}")
        continue

    func = getattr(talib, name)
    kwargs = {}
    if p1 is not None:
        kwargs["timeperiod"] = int(p1)
    if p2 is not None:
        kwargs["fastperiod"] = int(p2)
    if p3 is not None:
        kwargs["signalperiod"] = int(p3)

    # Apply to Close column
    ohlc_ta[f"{name}_{p1}"] = func(ohlc_ta["Close"], **kwargs)

# --- Step 4: View Result ---
print(ohlc_ta.tail())
